Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [12]:
x = tf.Variable([[1., 1.], [2., 2.]])
y = tf.Variable([1., 1. , 5])
loss = tf.nn.l2_loss([x, y])
sess = tf.Session()
with sess.as_default():
    tf.global_variables_initializer().run()
    print(x.eval())   # equivalent to session.run(b)
    print(loss.eval())


ValueError: Tried to convert 't' to a tensor and failed. Error: Shapes must be equal rank, but are 2 and 1
	From merging shape 0 with other shapes. for 'L2Loss_2/packed' (op: 'Pack') with input shapes: [2,2], [3].

In [3]:
x

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32_ref>

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
data_root = '/home/roman/data/nomnist' # Change me to store data elsewhere
pickle_file = data_root + '/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [12]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.005
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  #tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  #tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  # Matrix W of size KxY, where K is number of inputs, Y is number of outputs 
  # (in 1-hot encoding for multinomial regression)
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))  # Intercept
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  if beta > 0:
    loss += beta * (tf.nn.l2_loss(weights)) #  + tf.nn.l2_loss(biases)
    
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [14]:
num_steps = 3700

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, batch_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 35.561222
Training accuracy: 7.0%
Validation accuracy: 8.8%
Loss at step 500: 1.799416
Training accuracy: 82.8%
Validation accuracy: 79.6%
Loss at step 1000: 0.769601
Training accuracy: 82.0%
Validation accuracy: 81.2%
Loss at step 1500: 0.677496
Training accuracy: 81.2%
Validation accuracy: 81.2%
Loss at step 2000: 0.565497
Training accuracy: 85.9%
Validation accuracy: 80.7%
Loss at step 2500: 0.688383
Training accuracy: 82.0%
Validation accuracy: 81.5%
Loss at step 3000: 0.642052
Training accuracy: 83.6%
Validation accuracy: 81.2%
Loss at step 3500: 0.823378
Training accuracy: 78.9%
Validation accuracy: 80.9%
Test accuracy: 88.1%


In [23]:
batch_size = 256
hidden_size = 1024
input_size = image_size * image_size

def setup_lt(size_vec):
    assert(size_vec)
    res = []
    for index in range(len(size_vec) - 1):
        w = tf.Variable(tf.truncated_normal([size_vec[index], size_vec[index+1]]))
        b = tf.Variable(tf.zeros([size_vec[index+1]]))
        res.append((w, b))
    return res

def forward_prop(input, weights, apply_softmax = False):
    assert(weights)
    prev = input
    for index in range(len(weights) - 1):
      prev = tf.nn.relu(tf.matmul(prev, weights[index][0]) + weights[index][1])
    
    o = tf.matmul(prev, weights[-1][0]) + weights[-1][1]
    if apply_softmax:
        o = tf.nn.softmax(o)
    return o
    
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, input_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_beta = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = setup_lt((input_size, hidden_size, num_labels))

  # Training computation.
  logits = forward_prop(tf_train_dataset, weights)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  for w in weights:
    loss += tf_beta * (tf.nn.l2_loss(w[0]) + tf.nn.l2_loss(w[1]))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_prediction = forward_prop(tf_valid_dataset, weights, apply_softmax=True) 
  test_prediction = forward_prop(tf_test_dataset, weights, apply_softmax=True)

In [24]:
num_steps = 4700
beta = 0.0005

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_beta : beta}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, batch_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 484.721497
Training accuracy: 8.2%
Validation accuracy: 32.1%
Loss at step 500: 129.233612
Training accuracy: 82.8%
Validation accuracy: 81.1%
Loss at step 1000: 96.882797
Training accuracy: 84.4%
Validation accuracy: 81.4%
Loss at step 1500: 75.394951
Training accuracy: 82.8%
Validation accuracy: 83.0%
Loss at step 2000: 56.986263
Training accuracy: 87.9%
Validation accuracy: 84.7%
Loss at step 2500: 44.189522
Training accuracy: 85.9%
Validation accuracy: 84.3%
Loss at step 3000: 34.617283
Training accuracy: 84.8%
Validation accuracy: 85.2%
Loss at step 3500: 26.750139
Training accuracy: 89.1%
Validation accuracy: 84.7%
Loss at step 4000: 20.862314
Training accuracy: 90.2%
Validation accuracy: 86.3%
Loss at step 4500: 16.458727
Training accuracy: 87.5%
Validation accuracy: 87.2%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
num_steps = 4000
beta = 0.0005

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step % 100) * batch_size 
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_beta : beta}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, batch_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 480.971405
Training accuracy: 12.5%
Validation accuracy: 31.7%
Loss at step 500: 123.281479
Training accuracy: 89.1%
Validation accuracy: 79.8%
Loss at step 1000: 94.283295
Training accuracy: 95.3%
Validation accuracy: 81.5%
Loss at step 1500: 73.198013
Training accuracy: 97.7%
Validation accuracy: 82.6%
Loss at step 2000: 56.969032
Training accuracy: 99.2%
Validation accuracy: 82.5%
Loss at step 2500: 44.366539
Training accuracy: 99.6%
Validation accuracy: 83.0%
Loss at step 3000: 34.583382
Training accuracy: 98.8%
Validation accuracy: 82.5%
Loss at step 3500: 26.912333
Training accuracy: 100.0%
Validation accuracy: 83.0%
Loss at step 4000: 20.970167
Training accuracy: 100.0%
Validation accuracy: 83.3%
Loss at step 4500: 16.335474
Training accuracy: 100.0%
Validation accuracy: 83.7%
Test accuracy: 90.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
